## Data Preparation and Loading

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Dataset_assignment\ .zip

In [4]:
train_path = '/content/images/train'
test_path = '/content/images/final test'
val_path = '/content/images/validation'

train_data_path = os.listdir('/content/images/train')
test_data_path = os.listdir('/content/images/final test')
val_data_path = os.listdir('/content/images/validation')
print(f'label types: {train_data_path}')

label types: ['surprise', 'sad', 'happy', 'neutral', 'angry', 'fear', 'disgust']


In [5]:
# Preparing Data

def prepare_data(data_path, sub_data):
    rooms = []
    for item in data_path:
        all_rooms = os.listdir(str(sub_data) + '/' +item)

        for room in all_rooms:
            rooms.append((item, str(sub_data + '/' +item) + '/' + room))

    data = pd.DataFrame(data=rooms, columns=['tag', 'image'])
    return data

train_data = prepare_data(train_data_path, train_path).loc[:,['image','tag']]
test_data = prepare_data(test_data_path, test_path)
val_data = prepare_data(val_data_path, val_path)

print(f'Length of Train Data: {len(train_data)} \nLength of Test Data {len(test_data)} \nLength of Val Data {len(val_data)}')

Length of Train Data: 26921 
Length of Test Data 1900 
Length of Val Data 7066


In [6]:
def label_mapping(data):
    df = data.loc[:, ['image', 'tag']]
    print(f'Data shape: {df.shape}')

    label_map = {
        'angry': 0,
        'disgust': 1,
        'fear': 2,
        'happy': 3,
        'neutral': 4,
        'sad': 5,
        'surprise': 6
    }

    df['label'] = df['tag'].map(label_map)
    return df

train_ = label_mapping(train_data)
test_ = label_mapping(test_data)
val_ = label_mapping(val_data)


train_.to_csv('/content/drive/MyDrive/train.csv', index=False)
test_.to_csv('/content/drive/MyDrive/test.csv', index=False)
val_.to_csv('/content/drive/MyDrive/val.csv', index=False)

Data shape: (26921, 2)
Data shape: (1900, 2)
Data shape: (7066, 2)


In [7]:
train_df = pd.read_csv("/content/drive/MyDrive/train.csv", usecols=['image', 'label'])
test_df = pd.read_csv("/content/drive/MyDrive/test.csv", usecols=['image', 'label'])
val_df = pd.read_csv("/content/drive/MyDrive/val.csv", usecols=['image', 'label'])


print(f"Training: {len(train_df)} Testing: {len(test_df)} Testing: {len(val_df)}")

Training: 26921 Testing: 1900 Testing: 7066


In [15]:
val_df.head()

,image,label
0,/content/images/validation/surprise/26076.jpg,6
1,/content/images/validation/surprise/13101.jpg,6
2,/content/images/validation/surprise/23893.jpg,6
3,/content/images/validation/surprise/21072.jpg,6
4,/content/images/validation/surprise/25107.jpg,6


## Constructing Dataloader, Building model and Training model

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [9]:
class EmotionData:
    def __init__(self, data):
        self.data = data
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]

        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

In [10]:
train_dataset = EmotionData(train_df)
test_dataset = EmotionData(test_df)
val_dataset = EmotionData(val_df)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print("=" * 50)
print(f"Total No. of Batches: {len(train_dataloader)} \n")
for _, (features, labels) in enumerate(train_dataloader):
    print(f' \n Batch No: {_+1}, \n Data shape {features.shape} \n Labels {labels.shape}')
    print("=" * 50)
    break

Total No. of Batches: 421 

 
 Batch No: 1, 
 Data shape torch.Size([64, 3, 48, 48]) 
 Labels torch.Size([64])


In [24]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.4)
        )

        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.4)
        )

        self.fc1 = nn.Linear(512 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 7)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)

        x = x.view(-1, 512 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = self.fc3(x)

        return x

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [32]:
# hyperparameters, model, loss function, and optimizer.....

epochs = 100
learning_rate = 0.001

model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
def training_loop(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_outputs, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_outputs.append(outputs)
                val_labels.append(labels)

        val_outputs = torch.cat(val_outputs)
        val_labels = torch.cat(val_labels)
        _, preds = torch.max(val_outputs, 1)

        # Move tensors to the CPU before converting to numpy
        accuracy = accuracy_score(val_labels.cpu().numpy(), preds.cpu().numpy())

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {accuracy:.4f}')

        # Save the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_model.pth')


training_loop(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=epochs)

Epoch 1/100, Loss: 1.8171, Val Accuracy: 0.2504
Epoch 2/100, Loss: 1.7011, Val Accuracy: 0.3313
Epoch 3/100, Loss: 1.5147, Val Accuracy: 0.4023
Epoch 4/100, Loss: 1.4272, Val Accuracy: 0.4413
Epoch 5/100, Loss: 1.3724, Val Accuracy: 0.4557
Epoch 6/100, Loss: 1.3391, Val Accuracy: 0.4769
Epoch 7/100, Loss: 1.2999, Val Accuracy: 0.4850
Epoch 8/100, Loss: 1.2639, Val Accuracy: 0.5033
Epoch 9/100, Loss: 1.2322, Val Accuracy: 0.5235
Epoch 10/100, Loss: 1.2007, Val Accuracy: 0.5426
Epoch 11/100, Loss: 1.1652, Val Accuracy: 0.5358
Epoch 12/100, Loss: 1.1272, Val Accuracy: 0.5648
Epoch 13/100, Loss: 1.1059, Val Accuracy: 0.5592
Epoch 14/100, Loss: 1.0701, Val Accuracy: 0.5760
Epoch 15/100, Loss: 1.0441, Val Accuracy: 0.5941
Epoch 16/100, Loss: 1.0100, Val Accuracy: 0.5843
Epoch 17/100, Loss: 0.9794, Val Accuracy: 0.6006
Epoch 18/100, Loss: 0.9507, Val Accuracy: 0.5999
Epoch 19/100, Loss: 0.9285, Val Accuracy: 0.6124
Epoch 20/100, Loss: 0.9047, Val Accuracy: 0.6199
Epoch 21/100, Loss: 0.8757, V

In [34]:
# Load the best model(saved model)
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [35]:
# Testing on test data

model.eval()
test_outputs = []
test_labels = []
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_outputs.append(outputs.cpu())
        test_labels.append(labels.cpu())

test_outputs = torch.cat(test_outputs)
test_labels = torch.cat(test_labels)
_, preds = torch.max(test_outputs, 1)
test_accuracy = accuracy_score(test_labels.cpu().numpy(), preds.cpu().numpy())

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.6437
